# What is JuMP?

JuMP is an _modeling language_ for optimization problems, written in Julia. 

When solving an optimization problem, typically you start with something like this:

<!-- ![alt text](img/pooling_problem.png) -->

$$
\begin{align*}
\max_{x_1,x_2,x_3} \quad& x_1 + 2x_2 + 3x_3 \\
\text{s.t.}\quad& -x_1 + x_2 + x_3 \leq 20 \\
& x_1 -3x_2 + x_3 \leq 30 \\
& 0 \leq x_1 \leq 40 \\
& x_2, x_3 \geq 0.
\end{align*}
$$


To solve this, you might use a _solver_: a software implementation of an optimization algorithm. They typically want the problem specified in a much more opaque way:

```java
import ilog.concert.*;
import ilog.cplex.*;
public class Example {
 public static void main(String[] args) {
   try {
    IloCplex cplex = new IloCplex();
double[] lb = {0.0, 0.0, 0.0};
double[] ub = {40.0, Double.MAX_VALUE, Double.MAX_VALUE}; IloNumVar[] x = cplex.numVarArray(3, lb, ub);
    double[] objvals = {1.0, 2.0, 3.0};

cplex.addMaximize(cplex.scalProd(x, objvals));
    cplex.addLe(cplex.sum(cplex.prod(-1.0, x[0]),
                   cplex.prod( 1.0, x[1]),
                   cplex.prod( 1.0, x[2])), 20.0);
    cplex.addLe(cplex.sum(cplex.prod( 1.0, x[0]),
                   cplex.prod(-3.0, x[1]),
                   cplex.prod( 1.0, x[2])), 30.0);
if ( cplex.solve() ) {
cplex.out().println("Solution status = " + cplex.getStatus()); cplex.out().println("Solution value = " + cplex.getObjValue());
     double[] val = cplex.getValues(x);
     int ncols = cplex.getNcols();
     for (int j = 0; j < ncols; ++j)
       cplex.out().println("Column: " + j + " Value = " + val[j]);
    }
    cplex.end();
   }
   catch (IloException e) {
    System.err.println("Concert exception '" + e + "' caught");
} }
}
```

For larger, more complex problem, programming like this is:
* Time-consuming
* Difficult
* Hard to maintain/extend
* Error-prone

A modeling language (like JuMP) lets you code an optimization problem in a more natural way. It does the translation to the low-level solver format for you.

There are a number of modeling languages out there. Why JuMP?

* User-friendly
* Matches performance of competitors
* Solver-independent
* Easy to extend and take advantage of advanced features

For more information and a quick start guide to JuMP, take a look at [JuMP docs](https://jump.readthedocs.io/en/latest/index.html).

In this class, we will show how we can use JuMP for linear and integer optimization. 

# Installing JuMP

To install JuMP, just run

In [ ]:
Pkg.add("JuMP")

We already did this in the preassignment. To actually solve a problem, we will also need to install a solver package. There are 15+ options exposed in julia, each with support for different problem classes, different performance profiles, licensing requirements, etc. For the preassignment, we installed Gurobi, a best-of-breed linear/integer programming solver with a generous academic license.

In [ ]:
Pkg.add("Gurobi")

# A first example
Let's see how we translate a simple, 2 variable LP to JuMP code.

$$
\begin{align*}
\max_{x,y} \quad& x + 2y \\
\text{s.t.}\quad& x + y \leq 1 \\
& x, y \geq 0.
\end{align*}
$$

First, we load the JuMP and Gurobi libraries.

In [ ]:
using JuMP, Gurobi

Next, we construct a model object. This is a container for everything in our optimization problem: variables, constraints, solver options, etc.

In [ ]:
model = Model(solver=GurobiSolver())

Next, we define the two decision variables in our optimization problem. We will use the ``@variable`` macro (a fancy function, essentially). The first argument is the model object to attach the variable to, and the second specifies the variable name and any bounds.

In [ ]:
@variable(model, x >= 0)
@variable(model, y >= 0)

In [ ]:
model

We now add the single constraint of our problem using the ``@constraint`` macro. We write it algebraically, exactly as we see it above.

In [ ]:
@constraint(model, x + y <= 1)
model

We specify the objective function with the ``@objective`` macro.

In [ ]:
@objective(model, Max, x + 2y)

In [ ]:
model

To solve the optimization problem, call the ``solve`` function.

In [ ]:
solve(model)

We can now inspect the solution values and optimal cost.

In [ ]:
@show getvalue(x)
@show getvalue(y)
@show getobjectivevalue(model)

# Exercise

Code and solve the following optimization problem:

$$
\begin{align*}
\min_{x,y} \quad& 3x - y \\
\text{s.t.}\quad& x + 2y \geq 1 \\
& x \geq 0 \\
& 0 \leq y \leq 1.
\end{align*}
$$

In [ ]:
# Write your code here!

# Airline Network Revenue Management

<img style="max-width:100%; width:500px; height:auto" src="http://i.imgur.com/jeGwWET.png">

In the airline network revenue management problem we are trying to decide how many tickets for each origin-destination (O-D) pair to sell at each price level. The goal is to maximize revenue, and we cannot sell more tickets than there is demand for, or space on the planes for.

## Three Flight Problem

We'll start with a toy problem that has three origin-destination pairs, with two price classes for each pair. The three origin-destination pairs are BOS-MDW, MDW-SFO, or BOS-SFO via MDW. BOS stands for Boston, MDW is Chicago-Midway, and SFO is San Francisco. Each O-D pair has a "regular" and "discount" fare class. The data we will use is summarized as follows:

```
PLANE CAPACITY: 166

BOS-MDW
        Regular  Discount
Price:  428      190
Demand: 80       120

BOS-SFO
        Regular  Discount
Price:  642      224
Demand: 75       100

MDW-SFO
        Regular  Discount
Price:  512      190
Demand: 60       110
```

In [ ]:
nrm = Model(solver=GurobiSolver())

In [ ]:
@variables(nrm, begin 
    0 <= BOStoMDW_R <= 80
    0 <= BOStoMDW_D <= 120
    0 <= BOStoSFO_R <= 75
    0 <= BOStoSFO_D <= 100
    0 <= MDWtoSFO_R <= 60
    0 <= MDWtoSFO_D <= 110
end)
nrm

In [ ]:
@objective(nrm, Max, 428BOStoMDW_R + 190BOStoMDW_D +
                     642BOStoSFO_R + 224BOStoSFO_D +
                     512MDWtoSFO_R + 190MDWtoSFO_D)
nrm

In [ ]:
@constraint(nrm, BOStoMDW_R + BOStoMDW_D + 
                 BOStoSFO_R + BOStoSFO_D <= 166)
@constraint(nrm, MDWtoSFO_R + MDWtoSFO_D + 
                 BOStoSFO_R + BOStoSFO_D <= 166)
nrm

In [ ]:
status = solve(nrm)
status  # Should be `:Optimal`

In [ ]:
@show getvalue(BOStoMDW_R)
@show getvalue(BOStoMDW_D)
@show getobjectivevalue(nrm)

At this point, an exercise might be to extend this model by, say, adding another airport in this same fashion. I won't assign that, though, because it's a little tedious. It also doesn't scale well to problems with many airports. Instead, we can use JuMP's collections and summation notation to make compact, clear, and maintainable models for larger, more complex problems.

## Collections of variables and summation in JuMP

First, we would like to construct a _collection of variables_ all at once.  This is a very common idiom; for example, you might have a variable named ``x`` that is indexed from 1 to 10:

In [ ]:
m = Model()
@variable(m, x[1:10] >= 0)

The index sets are specified inside the ``[...]`` block. You can create multidimensional containers by specifying multiple index sets, separated by commas:

In [ ]:
@variable(m, y[1:10,["red","blue"]] <= 1)

For more complicated expressions, you can name the indices for the index sets and use them in the rest of the variable definition:

$$
i \leq z_{ij} \leq ub_j \;\;\; \forall i \in \{1,...,10\}, j \in \{i+1, ..., 10\}
$$

In [ ]:
ub = rand(10)
@variable(m, i <= z[i=1:10,j=(i+1):10] <= ub[j])

To specify conditions on the indexing, you can add conditionals inside the ``[...]`` block, separated by a semicolon:

In [ ]:
@variable(m, w[i=1:10, c=["red","blue"]; iseven(i) || c == "red"] >= 0)

Now that we can programatically create arrays of variables, we would like to be able to use them to full-effect in the constraints of our problem. That is, we want a way to express multi-dimensional summations, with conditionals. To do this, we use the ``sum(...)`` construction. The first argument is the ''inner loop'' of the summation, the index sets are specified after a ``for``, and any conditionals are stated following an ``if`` (similar to variable definition, but with a slightly different syntax).

$$ \sum _{i = 1}^{10} x_i \leq 1$$

In [ ]:
@constraint(m, sum(x[i] for i in 1:10) <= 1)

$$ 
\begin{equation}
\sum_{\substack{i\in\{1,...,10\}\\
                c\in\{"red","blue"\}}}
       coef(c) \cdot y_{ic} = 1
\end{equation}
$$

In [ ]:
coef = Dict("red" => 2, "blue" => 3)
@constraint(m, sum(coef[c]*y[i,c] for i in 1:10, c in ["red","blue"]) == 1)

$$ 
\begin{equation}
\sum_{i = 1}^{10} \sum_{j = i+1}^{10} 
       i \cdot j \cdot z_{ij} \leq
\sum_{\substack{i\in\{1,...,10\},
                c\in\{"red","blue"\} \\
                \text{s.t. } iseven(i) \text{ or } c = "red"}}
       i^2 \cdot w_{ic} 
\end{equation}
$$

In [ ]:
@constraint(m, sum(i*j*z[i,j] for i in 1:10, j in (i+1):10) <=
               sum(i^2*w[i,c] for i in 1:10, c in ["red","blue"] if iseven(i) || c == "red"))

## Extension to more airports

Now let's return to the network revenue management example and attempt to rewrite it in a generic way that scales to any number of airports. 

First, let's create some random data for our problem.

In [ ]:
# Set the random seed to ensure we always
# get the same stream of 'random' numbers
srand(1234)  

# Let's create a vector of symbols, one for each airport
airports = [:BOS, :MDW, :SFO, :YYZ]
num_airport = length(airports)

# We'll also create a vector of fare classes
classes = [:REG, :DIS]

# All the demand and price data for each triple of
# (origin, destination, class) will be stored in
# 'dictionaries', also known as 'maps'.
demand = Dict()
prices = Dict()

# Generate a demand and price for each pair of airports
# To keep the code simple we will generate info for
# nonsense flights like BOS-BOS and SFO-SFO, but they
# won't appear in our final model.
for origin in airports, dest in airports
    # Generate demand:
    #  - Regular demand is Uniform(50,90)
    #  - Discount demand is Uniform(100,130)
    demand[(origin,dest,:REG)] = rand(50:90)    
    demand[(origin,dest,:DIS)] = rand(100:130)
    # Generate prices:
    #  - Regular price is Uniform(400,700)
    #  - Discount price is Uniform(150,300)
    prices[(origin,dest,:REG)] = rand(400:700)
    prices[(origin,dest,:DIS)] = rand(150:300)
end

# Finally set all places to have the same capacity
plane_cap = rand(150:200)

# Lets look at a sample demand at random
@show demand[(:BOS,:YYZ,:REG)]

Now let's build the model. We will have our decision variable ``x`` indexed by three things:

1. Origin
2. Destination
3. Class

The upper bound (the demand for each) will vary accordingly.

In [ ]:
nrm2 = Model(solver=GurobiSolver())

@variable(nrm2, 0 <= x[o=airports,
                       d=airports,
                       c=classes; o!=d] <= demand[(o,d,c)])
nrm2

The objective is to maximize the profit we make, summing over each ticket set:

In [ ]:
@objective(nrm2, Max, sum(prices[(o,d,c)]*x[o,d,c] for 
    o in airports, d in airports, c in classes if o != d))
nrm2

Our first set of constraints enforces that all the legs leaving the hub airport must not oversell the plane capacity:

In [ ]:
for d in airports
    if d != :MDW
        println("Adding constraint for hub (MDW) to $d")
        @constraint(nrm2, 
            sum(x[o,d,c] for o in airports, c in classes if o!=d) <= plane_cap)
    end
end
nrm2

Now, as an exercise, complete this model by adding constraints that each flight _to_ the hub is not oversold.

In [ ]:
# Constraints here!


# Now solve the model


## Sudoku

![Sudoku](http://upload.wikimedia.org/wikipedia/commons/f/ff/Sudoku-by-L2G-20050714.svg)

**Sudoku** is a number puzzle played on a 9x9 grid. The challenge is to place a digit between 1 and 9 inclusive in each empty cell, such that the completed grid obeys the following rules:

* Each row contains the numbers 1 to 9 once and only once.
* Each column contains the numbers 1 to 9 once and only once.
* Each 3x3 subgrid contains the numbers 1 to 9 once and only once.

In [ ]:
init_vals = [
5 3 0 0 7 0 0 0 0
6 0 0 1 9 5 0 0 0
0 9 8 0 0 0 0 6 0
8 0 0 0 6 0 0 0 3
4 0 0 8 0 3 0 0 1
7 0 0 0 2 0 0 0 6
0 6 0 0 0 0 2 8 0
0 0 0 4 1 9 0 0 5
0 0 0 0 8 0 0 7 9
]

using PyPlot
# Displays a sudoku matrix. If an initial state is provided, 
# then the new numbers are displayed in red. 
function display_sudoku(sudoku, initial_state=zeros(9,9))
    fig, ax = subplots()
    ax[:axis]("off")
    # Make sudoku basic 9x9 grid.
    for i in 0:9
        plot([i,i], [0, 9], color = "black")
        plot([0, 9], [i,i], color = "black")
    end
    # Make thicker lines to separate the 3x3 subgrids.
    for i in 0:3:9
        plot([i,i], [0,9], color = "black", linewidth = 3)
        plot([0,9], [i,i], color = "black", linewidth = 3)
    end
    # Display the values at the right square.
    for i in 0:8
        for j in 0:8 
            value = Int(sudoku[9-j,i+1])
            old_value = initial_state[9-j,i+1]
            # If an initial state is not provided (set to zero) or the value of the 
            # square did not change, the color is set to black. Otherwise, it becomes red.
            col = (sum(initial_state) == 0 || old_value == value) ? "black" : "red"
            if value > 0 # Omit zero values  
                text(i + 0.35, j + 0.33, value, size = 16, color = col)
            end
        end
    end
end
display_sudoku(init_vals)

The most natural formulation of this problem would probably be something like

$$x_{i,j} \in \{1, 2, \dots, 9\}$$

which is of course something we can do with integer programming:

$$1 \leq x_{i,j} \leq 9, ~ integer$$

The challenge now is the constraints. One observation is that the numbers 1 to 9 add up to 45, so we could try something like:

$$ \sum_{j=1}^9 x_{i,j} = 45 $$

In [ ]:
using JuMP, Gurobi
function sudoku1()
    sudoku = Model(solver=GurobiSolver(OutputFlag=0))
    @variable(sudoku, 1 ≤ x[1:9,1:9] ≤ 9, Int)
    
    @constraints(sudoku, begin
        rows[i=1:9], sum(x[i,:]) == 45
        cols[j=1:9], sum(x[:,j]) == 45
        subgrid[i=1:3:7,j=1:3:7], sum(x[i:i+2,j:j+2]) == 45
    end)
    
    for i in 1:9, j in 1:9
        if init_vals[i,j] != 0
            @constraint(sudoku, x[i,j] == init_vals[i,j])
        end
    end
    
    solve(sudoku)
    
    getvalue(x)
end
@time soln1 = sudoku1();

In [ ]:
display_sudoku(soln1, init_vals)

Well that didn't work. We might be able to make this work, but we'd need many more constraints. Instead, let's change our **variables**: $x_{i,j,k} = 1$ iff the number $k$ will appear in cell $(i,j)$. We can now use our 0-1 integer programming knowledge to model the problem. Consider the "row" constraints: we want each number to appear once and only once. This is equivalent to saying that

$$\sum_{j=1}^9 x_{i,j,k} = 1 \quad \forall i, k$$

We can now model this as a $9\times 9\times 9$ dimensional problem - thats a lot of binary variables, surely Gurobi won't like that!

In [ ]:
function sudoku2()
    sudoku = Model(solver=GurobiSolver())

    @variable(sudoku, x[i=1:9, j=1:9, k=1:9], Bin)

    @constraints(sudoku, begin
        # Constraint 1 - Exactly one value appears in each cell.
        cell[i=1:9, j=1:9], sum(x[i,j,:]) == 1
        # Constraint 2 - Each value appears in each row exactly once.
        row[i=1:9, k=1:9], sum(x[i,:,k]) == 1
        # Constraint 3 - Each value appears in each column exactly once.
        # Try to write the constraints for the columns!
        
        # Constraint 4 - Each value appears in each 3x3 subgrid exactly once.
        subgrid[i=1:3:7,j=1:3:7,val=1:9], sum(x[i:i+2,j:j+2,val]) == 1
    end)

    # Fix given values. 
    for row in 1:9, col in 1:9
        if init_vals[row,col] != 0
            @constraint(sudoku, x[row, col, init_vals[row, col]] == 1)
        end
    end
    
    solve(sudoku)
    
    getvalue(x)
end
@time soln2 = sudoku2();

In [ ]:
# soln2 is a 9x9x9 array with values 0 or 1. 
# First we need to transform it to a 9x9 matrix with the right values 1,...,9.
soln2_array = sum(i * soln2[:,:,i] for i in 1:9)
display_sudoku(soln2_array, init_vals)

## Presolving the Problem
Can you see the lines
```
Optimize a model with 354 rows, 729 columns and 2946 nonzeros
Presolve removed 354 rows and 729 columns
```
? Gurobi was able to use logic to deduce the value of every variable - no linear relaxation required! This "magic" is actually how a human might solve it. Consider the following:

We know that $x_{2,6,5}$ is fixed at 1 because it is one of the provided values. So we can actually replace $x_{2,6,5}$ wherever it appears in the constraints with the constant 1:

"The value 5 must appear in row 2":
$$x_{2,1,5} + x_{2,2,5} + x_{2,3,5} + x_{2,4,5} + x_{2,5,5} + x_{2,6,5} + x_{2,7,5} + x_{2,8,5} + x_{2,9,5} = 1$$
$$\rightarrow x_{2,1,5} + x_{2,2,5} + x_{2,3,5} + x_{2,4,5} + x_{2,5,5} + 1 + x_{2,7,5} + x_{2,8,5} + x_{2,9,5} = 1$$
$$\rightarrow x_{2,1,5} + x_{2,2,5} + x_{2,3,5} + x_{2,4,5} + x_{2,5,5} + x_{2,7,5} + x_{2,8,5} + x_{2,9,5} = 0$$

"The value 5 must appear in column 6":
$$x_{1,6,5} + x_{2,6,5} + x_{3,6,5} + x_{4,6,5} + x_{5,6,5} + x_{6,6,5} + x_{7,6,5} + x_{8,6,5} + x_{9,6,5} = 1$$
$$x_{1,6,5} + 1 + x_{3,6,5} + x_{4,6,5} + x_{5,6,5} + x_{6,6,5} + x_{7,6,5} + x_{8,6,5} + x_{9,6,5} = 1$$
$$x_{1,6,5} + x_{3,6,5} + x_{4,6,5} + x_{5,6,5} + x_{6,6,5} + x_{7,6,5} + x_{8,6,5} + x_{9,6,5} = 0$$

and so on. Because all those other variables can only be 0 or 1, and their sum is 0, they must all be 0. Thus Gurobi presolve can perform the following procedure:
1. Replace all the fixed values with constants
2. Use constraints to fix variables, e.g. at 0 (or 1)
3. Go to 1 unless step 2 did nothing.

A small problem arises when there a multiple solutions to the problem - a random selection has to be made. Gurobi will treat this case as "trying to find a feasible solution" - it will fix a variable, and follow the implications through.